In [16]:
import got3 as got

#Fetch a number of tweets for a certain keyword for a certain date span.
#keyword is a query like "elizabeth warren"
#max_tweets is the max number of tweets to fetch
def fetch_tweets(keyword, start_date, end_date, max_tweets=10):
    #Set the criteria
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(keyword).\
    setSince(start_date).setUntil(end_date).setMaxTweets(max_tweets)
    
    #Fetch the tweets
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)
    
    #Make it a list
    tweet_list=[]
    for tweet in tweets:
        tweet_list.append(tweet.text)

    return tweet_list

tweet_list = fetch_tweets("elizabeth warren", "2018-01-11", "2018-02-12")

In [17]:
#Data cleansing

import numpy as np
import re

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)        
    return input_txt


def clean_tweets(lst):
    # remove twitter Return handles (RT @xxx:)
    lst = np.vectorize(remove_pattern)(lst, "RT @[\w]*:")
    # remove twitter handles (@xxx)
    lst = np.vectorize(remove_pattern)(lst, "@[\w]*")
    # remove URL links (httpxxx)
    lst = np.vectorize(remove_pattern)(lst, "https?://[A-Za-z0-9./]*")
    # remove special characters, numbers, punctuations (except for #)
    lst = np.core.defchararray.replace(lst, "[^a-zA-Z#]", " ")
    
    return lst

tweet_list_clean = clean_tweets(tweet_list)
tweet_list_clean

array(["Wow I'm impressed Pocahantis. So you earned your paycheck for a change? About time you did something",
       ' DNC  DNCWomen # Dossier # FISAMemo # deepstategate  NBCNews  ABC # DeepStatePanic # OlympicGames # OlympicWinterGames  IsraelUSAforevr  SenFeinstein  SenWarren # Senate  realDonaldTrump  JudicialWatch  JulianAssange # MorningJoe',
       'How many times have we heard this excuse? I say Elizabeth Warren is a sell-out. She lost my respect w her pandering mercilessly for HRC. The only way CHANGE us going to happen is a united front against Lobbyists, and I dont see that happening anytime soon.',
       'Amount paid. $0.00 Says it all.',
       "Elizabeth Warren ANNIHILATES Jeff Sessions' Career Of Bigotry _GkeiY via  YouTube",
       'What is your response to this? Or are you too afraid of this question?  SenatorHassan  SenatorShaheen  SenWarren  PattyMurray  NHDems  NHGOP …',
       'In time.....the truth will shake itself down...',
       'I was always pro life ,but th

In [22]:
#Perform sentiment analysis on a list of tweets
import requests
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def get_average_sentiment(tweet_list):
    analyser = SentimentIntensityAnalyzer()

    scores = []
    for tweet in tweet_list:
        scores.append(analyser.polarity_scores(tweet)['compound'])

    return sum(scores)/len(scores)
    
get_average_sentiment(tweet_list_clean)

0.07729

In [23]:
#Sentiment score for a given keyword on a given day.

def get_sentiment_for_keyword(keyword, start_date, end_date):
    tweet_list = fetch_tweets(keyword, start_date, end_date)
    tweet_list_clean = clean_tweets(tweet_list)
    return get_average_sentiment(tweet_list_clean)

In [48]:
get_sentiment_for_keyword("bud light", "2018-01-01", "2018-12-01")

0.07428